In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
%matplotlib notebook

In [3]:
def binary_search(sorted_list, target, ind):  # returns index of first value >=target
    # sorted list is assumed to contain (x,y,z) coordinates. search is done based on ind coordinate (i.e. ind = 0 means x)
    # returns len(sorted_list) if not found
    lo = 0
    hi = len(sorted_list) - 1
    while lo < hi - 1:
        mid = (lo + hi) // 2
        if sorted_list[mid][ind] >= target:
            hi = mid
        else:
            lo = mid
    if target > sorted_list[hi][ind]:
        return len(sorted_list)
    return lo if sorted_list[lo][ind] >= target else hi

def smooth_data(data,kernel_size = 20):#smooth some time series data with kernel
    kernel = np.ones(kernel_size)/kernel_size
    return np.convolve(data,kernel,mode='same')

In [4]:
class Connectome:
    # attr
    # node_locations: maps node names to (x,y,z) coords
    # names: list of node names
    # coordinates: list of (x,y,z) coords, sorted by increasing x, with order corresponding to names
    # ie node names[i] has coordinate coordinates[i]
    def __init__(self, G):
        self.G = G
        self.n_nodes = G.number_of_nodes()
        
        self.types_dict = dict()# "coarse" label mapped to one-hot index
        self.node_types = nx.get_node_attributes(G,'coarse')#node name mapped to coarse type
        unique_types = set(self.node_types.values())
        ind = 0
        for soma_type in unique_types:
            self.types_dict[soma_type] = ind
            ind+=1
        
        
        Xs = nx.get_node_attributes(G, 'x')
        Ys = nx.get_node_attributes(G, 'y')
        Zs = nx.get_node_attributes(G, 'z')
        
        self.node_dict = dict()
        self.lo_bound = []  # [min x, min y, min z]
        self.hi_bound = []  # [max x, max y, max z]
        for node in list(G.nodes):
            try:
                if float(Xs[node]) > 80:
                    continue
                self.node_dict[node] = [float(Xs[node]), float(Ys[node]), float(Zs[node])]
                if len(self.lo_bound) == 0:
                    self.lo_bound = self.node_dict[node][:]
                    self.hi_bound = self.node_dict[node][:]
                else:
                    for ind in range(3):
                        self.lo_bound[ind] = min(self.lo_bound[ind], self.node_dict[node][ind])
                        self.hi_bound[ind] = max(self.hi_bound[ind], self.node_dict[node][ind])
            except KeyError:
                pass
        
        
        self.coordinates = []  # extract [x,y,z] nodes
        self.names = []  # extract names
        for node, coord in self.node_dict.items():
            self.coordinates.append(coord)
            self.names.append(node)
        self.coordinates = np.asarray(self.coordinates)
        self.names = np.asarray(self.names)
        reorder = self.coordinates[:, 0].argsort()
        self.coordinates = self.coordinates[reorder]
        self.names = self.names[reorder]

    #use this to 3d plot null and empirical networks
    def plot_nodes(self,node_names,edges=None,display_labels=False):
        #node_names - all nodes to be plotted
        #edges- list of edges. if None, will be taken from empirical network

        if len(node_names)==0:
            return False


        if edges==None:
            edges = []
            seen_nodes = set()
            for node in node_names:
                for neighbor in self.G.neighbors(node):
                    if neighbor in seen_nodes:
                        edges.append([node, neighbor])
                seen_nodes.add(node)

        x_plots = [self.node_dict[name][0] for name in node_names]
        y_plots = [self.node_dict[name][1] for name in node_names]
        z_plots = [self.node_dict[name][2] for name in node_names]
        fig = plt.figure(figsize=(5, 5))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')

        ax.scatter(x_plots, y_plots, z_plots, s=30, color='blue')

        if display_labels:
            for node in node_names:
                ax.text(*self.node_dict[node],node,size=10,zorder=1,color='k')

        for node1, node2 in edges:
            plt_args = [[self.node_dict[node1][i], self.node_dict[node2][i]] for i in range(3)]
            ax.plot(*plt_args)
        plt.show()

        return True

    def plot_random_cube(self,cube_dim):
        success = False
        while not success:
            bounds = self.get_random_bounds(cube_dim)
            names, _ = self.get_cube(bounds,cube_dim)
            success = self.plot_nodes(names)

    # get random bounds for a cube to parse
    def get_random_bounds(self, cube_dim):
        center = [np.random.uniform(self.lo_bound[ind], self.hi_bound[ind]) for ind in range(3)]
        return [coord - (cube_dim / 2) for coord in center]

    def get_restricted_bounds(self,cube_dim,restriction,above):
        #get bounds, not past z-coordinate give by restriction
        #if above==True, bounds will be given such that cube lies above restriction. If above==False, cube will lie below restriction
        center = [np.random.uniform(self.lo_bound[ind], self.hi_bound[ind]) for ind in range(2)]
        if above:
            center.append(np.random.uniform(restriction+(cube_dim/2),self.hi_bound[2]))
        else:
            center.append(np.random.uniform(self.lo_bound[2],restriction - (cube_dim / 2)))
        return [coord - (cube_dim / 2) for coord in center]

    # finds cube in network and returns nodes in cube
    def get_cube(self, bounds, cube_dim):
        # bounds = [xbound,ybound,zbound], cube dim is side size of square
        # returns names, locations sorted by ascending z coordinates (ndarrays)
        lo_ind, hi_ind = 0, 0
        locations = self.coordinates
        names = self.names

        for ind in range(3):
            lo_ind = binary_search(locations, bounds[ind], ind)
            if lo_ind == -1:
                #                 print('no nodes in range')#no nodes left
                return np.array([]), np.array([])
            hi_ind = binary_search(locations, bounds[ind] + cube_dim, ind)
            if hi_ind <= lo_ind:
                #                 print('no nodes in range')#no nodes left
                return np.array([]), np.array([])
            locations = locations[lo_ind:hi_ind]
            names = names[lo_ind:hi_ind]
            if (ind != 2):
                reorder = locations[:, ind + 1].argsort()
                locations = locations[reorder]
                names = names[reorder]
        return names, locations

    # parse cube into model inputs and outputs
    def parse_cube(self, names, locations, dim, keep_all_inputs,incl_prob=.5):
        # takes nodes and parses into (dim x dim x dim) matrix, then extracts degree structure and adjacency matrix
        # names is ndarray of node names, locations ndarray of coordinates
        # formated like output of get_subgraph (ie sorted by ascending z coordinates)
        # dim is size of cube, in terms of neurons per side
        # return flattened arrays degrees,adjacencies,name which are representations of the degree structure and adjacency matrix and a list of names in order

        if len(names) < dim ** 3:
            #             print('not enough nodes to parse')
            return np.array([]), np.array([]), np.array([])

        parsed_names = []
        degrees = np.zeros((dim ** 3)+1)
        degrees[-1] = 1#use degrees
        adj_matrix = [[0 for _ in range(row_len)] for row_len in
                      range(dim ** 3)]  # 2d list indexed by [higher node index],[lower node index]
        node_indices = {}
        for bott_ind in range(0, dim ** 3, dim ** 2):
            layer_names = names[bott_ind:bott_ind + dim ** 2]
            layer_locs = locations[bott_ind:bott_ind + dim ** 2]
            reorder = layer_locs[:, 1].argsort()  # sort by y
            layer_names = layer_names[reorder]
            layer_locs = layer_locs[reorder]
            for lo_ind in range(0, dim ** 2, dim):
                row_names = layer_names[lo_ind:lo_ind + dim]
                row_locs = layer_locs[lo_ind:lo_ind + dim]
                reorder = row_locs[:,0].argsort()  # sort by x
                row_names = row_names[reorder]
                row_locs = row_locs[reorder]
                for col in range(dim):
                    node_index = bott_ind + lo_ind + col
                    # get neighbors, update node_info and edges
                    for neighbor in self.G.neighbors(row_names[col]):
                        if neighbor in node_indices:
                            degrees[node_index] += 1
                            degrees[node_indices[neighbor]] += 1
                            adj_matrix[node_index][node_indices[neighbor]] = 1
                    node_indices[row_names[col]] = node_index
                    parsed_names.append(row_names[col])
        
        #1=included, 0=not (last index)
        clustering_input = np.asarray([np.random.uniform(),0])
        soma_input = np.random.uniform(size=((dim**3*len(self.types_dict))+1))
        soma_input[-1] = 0
        if keep_all_inputs or np.random.uniform()<incl_prob:#use clustering?
            clustering_input[1] = 1#use clustering!
            edge_list = []
            for n1 in range(dim**3):
                for n2 in range(n1):
                    if adj_matrix[n1][n2]==1:
                        edge_list.append((parsed_names[n1],parsed_names[n2]))
            subg = nx.Graph()
            subg.add_nodes_from(parsed_names)
            subg.add_edges_from(edge_list)
            clustering_input[0] = nx.algorithms.cluster.transitivity(subg)

        if keep_all_inputs or np.random.uniform()<incl_prob:#use soma types?
            soma_input = np.zeros(((dim**3*len(self.types_dict))+1))
            soma_input[-1] = 1#use soma types!
            for node_ind in range(len(parsed_names)):
                one_hot_ind = self.types_dict[self.node_types[parsed_names[node_ind]]]
                soma_input[len(self.types_dict)*node_ind+one_hot_ind] = 1

        if np.random.uniform()>incl_prob and not keep_all_inputs:#dont use degrees?
            degrees = np.random.uniform(size=((dim**3)+1))
            degrees[-1] = 0#dont use degrees
        
        features = np.concatenate((degrees,clustering_input,soma_input))
        adj_matrix = [val for adj_row in adj_matrix for val in adj_row]
        
        return features, np.asarray(adj_matrix,dtype=np.float64),np.asarray(parsed_names)

    # gets and parses subgraph in cube with given bounds
    def get_and_parse(self, cube_dim, parse_dim, bounds,keep_all_inputs):
        # return flattened ndarrays degrees,adjacencies, which are representations of the degree structure and adjacency matrix
        names, locations = self.get_cube(bounds, cube_dim)
        features, adjacencies,names = self.parse_cube(names, locations, parse_dim,keep_all_inputs)
        return features, adjacencies,np.asarray(names)

    # get flattened degrees, flattened adjacencies of a random cube
    def random_get_and_parse(self, cube_dim, parse_dim,keep_all_inputs=False):
        # random bound generation continues until viable one found
        # cube_dim- side size of cube used in subnetwork
        # parse dim- side dimension of 3d matrix to parse subgraph into
        # return flattened ndarrays degrees,adjacencies, which are representations of the degree structure and adjacency matrix
        while True:
            bounds = self.get_random_bounds(cube_dim)
            features, adjacencies,names = self.get_and_parse(cube_dim, parse_dim, bounds,keep_all_inputs)

            if len(features) != 0:
                return features, adjacencies,names

    # get flattened degrees, flattened adjacencies of a random cube with restricted bounds
    def restricted_get_and_parse(self,cube_dim,parse_dim,train_split,is_test,keep_all_inputs=False):
        restriction = self.lo_bound[2]+((self.hi_bound[2]-self.lo_bound[2])*train_split)
        while True:
            bounds = self.get_restricted_bounds(cube_dim,restriction,is_test)#generate bounds above(is_test) or below(!is_test) restriction line
            features, adjacencies, names = self.get_and_parse(cube_dim, parse_dim, bounds,keep_all_inputs)

            if len(features) != 0:
                return features, adjacencies, names

    #generates batches for train and validation
    def batch_generator(self,cube_dim,parse_dim,train_split,is_test,batch_size=32):
        #randomly generate examples for training and testing
        #train_split is fraction of 3d space that are used for testing, rest are used for validation
        #train examples will be generated in the lower fraction of cube, corresponding to train_split
        #validation examples will use upper fraction of cube (1-train_split)
        n_nodes = int(parse_dim**3)#number of nodes per cube
        input_size = (parse_dim**3)+1+2+((parse_dim**3)*len(self.types_dict))+1
        
        while True:
            all_features = np.empty((batch_size,input_size))
            all_labels = np.empty((batch_size,int(n_nodes*(n_nodes-1)/2)))
            for i in range(batch_size):
                features, labels,_ = self.restricted_get_and_parse(cube_dim, parse_dim,train_split,is_test)
                all_features[i] = features
                all_labels[i] = labels
            yield all_features,all_labels
        
    def chung_lu_batch_generator(self,cube_dim,parse_dim,batch_size = 32):
        input_size = (parse_dim**3)+1+2+((parse_dim**3)*len(self.types_dict))+1
        
        degrees = []
        for _ in range(1000):
            deg,a,n = self.random_get_and_parse(cube_dim,parse_dim)
            degrees+=[d for d in deg[:parse_dim**3]]
        degrees = np.asarray(degrees)
        while True:
            n_nodes = parse_dim**3
            all_features = np.empty((batch_size,input_size))
            all_labels = np.empty((batch_size,int((n_nodes*(n_nodes-1))/2)))
            for i in range(batch_size):
                sample_degrees = np.random.choice(degrees,size=(n_nodes,))
                n_edges = np.sum(sample_degrees)/2
                labels = []#adjacency scores for node pairs
                for n1 in range(n_nodes):
                    for n2 in range(n1):
                        score = float(sample_degrees[n1]*sample_degrees[n2])/float((2*n_edges)-sample_degrees[n1])
                        labels.append(min(1.0,score))
                flag_and_clust = np.asarray([1,np.random.uniform(),0])#flag for degrees and random clustering val/flag
                soma_input = np.random.uniform(size=((parse_dim**3)*len(self.types_dict)+1))
                soma_input[-1] = 0
                features = np.concatenate((sample_degrees,flag_and_clust,soma_input))
                all_features[i] = np.asarray(features)
                all_labels[i] = np.asarray(labels)
            yield all_features,all_labels
        

In [57]:
class NullGenerator:
    
    def __init__(self,connectome,cube_dim,parse_dim):
        
        self.connectome = connectome
        
        self.parse_dim = parse_dim
        self.cube_dim = cube_dim
        
        self.model = tf.keras.Sequential([
            tf.keras.layers.Dense(64,activation=tf.keras.layers.LeakyReLU()),
            tf.keras.layers.Dense(32,activation=tf.keras.layers.LeakyReLU()),
            tf.keras.layers.Dense((parse_dim**3)*(parse_dim**3-1)/2,use_bias=False)
        ])
    
    def train(self,train_split=.5,batch_size=32,chung_epochs=0,train_epochs=500,steps_per_epoch=100,validation_steps=2,verbose=1,plot_metrics = True):
        #train model. outputs train history
        self.model.compile(
            optimizer='adam',
            loss = tf.keras.losses.MeanSquaredError(),
            metrics = ['accuracy']
        )
        print("training on Chung-Lu")
        chung_generator = self.connectome.chung_lu_batch_generator(self.cube_dim,self.parse_dim,batch_size=batch_size)
        self.model.fit(x=chung_generator,epochs=chung_epochs,steps_per_epoch=steps_per_epoch,verbose=verbose)
        print("Done on Chung-Lu")
        print("training on empirical")
        train_generator = self.connectome.batch_generator(self.cube_dim,self.parse_dim,train_split,False)
        validation_generator = self.connectome.batch_generator(self.cube_dim,parse_dim,train_split,True)
        
        hist = self.model.fit(x=train_generator, validation_data=validation_generator, epochs=train_epochs, validation_steps = validation_steps, steps_per_epoch=steps_per_epoch, verbose=verbose)
        
        if plot_metrics:
            plt.figure(figsize=(5,5))
            plt.plot(list(range(n_epochs)),hist.history['accuracy'],label='accuracy')
            plt.plot(list(range(n_epochs)),hist.history['loss'],label='loss')
            plt.plot(list(range(n_epochs)),hist.history['val_accuracy'],label='validation accuracy')
            plt.plot(list(range(n_epochs)),hist.history['val_loss'],label='validation loss')
            plt.legend(loc='upper right')
            plt.xlabel('time step')
            plt.ylabel('value')
            plt.show()
        return hist
        
    def cube_test(train_split, threshold=None):
        features, adjacencies, names, locations = [],[],[],[]
        while len(adjacencies)==0:
            features,adjacencies,names = self.connectome.restricted_get_and_parse(self.cube_dim,self.parse_dim,train_split,True)
        features = np.expand_dims(degrees,0)
        predicted_adj = self.model(degrees)[0]
        
        #find the optimal threshold such that the number of predicted edges matches that of the empirical network
        if threshold==None:
            sorted_adj = np.copy(predicted_adj)
            sorted_adj = np.sort(sorted_adj)
            if sum(adjacencies)==0:
                threshold = sorted_adj[-1]+1.0
            else:
                threshold = sorted_adj[len(sorted_adj)-int(sum(adjacencies))]
        
        loss = tf.keras.losses.MeanSquaredError()(adjacencies,predicted_adj)
        print(f"MSE loss: {loss}")
        predicted_edges = []
        for i in range(1,len(names)):
            adj_start = i*(i-1)/2
            for prev_node in range(i):
                adj_ind = int(prev_node+adj_start)
                if predicted_adj[adj_ind]>=threshold:
                    predicted_edges.append([names[i],names[prev_node]])
        print("predicted graph: ")
        self.connectome.plot_nodes(names,predicted_edges)
        print("empirical graph: ")
        self.connectome.plot_nodes(names)
        
        #plot ROC curve
        aroc = 0.0
        TPRs = [0.0]
        FPRs = [0.0]
        pred = predicted_adj.numpy()
        reorder = pred.argsort()
        pred = pred[reorder]
        adjacencies = adjacencies[reorder]
        n_pos = 0
        total_pos = sum(adjacencies)
        total_neg = len(adjacencies)-total_pos
        for i in range(len(pred)-1,-1,-1):
            if adjacencies[i]==1:#positive
                n_pos+=1
            TPRs.append(n_pos/total_pos)
            FPRs.append((len(pred)-i-n_pos)/total_neg)
            aroc+=TPRs[-1]*(FPRs[-1]-FPRs[-2])
        plt.figure(figsize=(5,5))
        plt.plot(FPRs,TPRs)
        plt.xlabel('FPR')
        plt.ylabel('TPR')
        plt.title('ROC curve')
        plt.show()
        print(f'AROC: {aroc}')
    
    #get average loss and over ROC curve for niter predictions of the algo
    def average_test(self,train_split,niter=5000):
        all_losses, all_adj, all_pred_adj = [],[],[]
        for iter in range(niter):
            features, adjacencies, names, locations = [],[],[],[]
            while len(adjacencies)==0:
                features,adjacencies,names = self.connectome.restricted_get_and_parse(self.cube_dim,self.parse_dim,train_split,True,keep_all_inputs=True)
            features = np.expand_dims(features,0)
            predicted_adj = self.model(features)[0]
            all_losses.append(tf.keras.losses.MeanSquaredError()(adjacencies,predicted_adj))
            all_pred_adj.append(predicted_adj.numpy())
            all_adj.append(adjacencies)
        
        #flatten and reorder arrays
        all_adj = np.asarray(all_adj)
        all_adj = all_adj.flatten()
        all_pred_adj = np.asarray(all_pred_adj)
        all_pred_adj = all_pred_adj.flatten()
        reorder = all_pred_adj.argsort()
        all_adj = all_adj[reorder]
        all_pred_adj = all_pred_adj[reorder]
        
        self.plot_ROC(all_adj,all_pred_adj,f'Model ROC over {niter} iterations')
        print(f'average loss: {np.mean(all_losses)}')
    
    
    def average_chung_lu(self,train_split,niter=5000):
        #this shit mad buggy for this parse function, feex it!
        all_adj, all_pred_adj = [],[]
        for iter in range(niter):
            degrees, adjacencies, names, locations = [],[],[],[]
            while len(adjacencies)==0:
                features,adjacencies,names = self.connectome.restricted_get_and_parse(self.cube_dim,self.parse_dim,train_split,True)
            degrees = features[:self.parse_dim**3]#extract degrees
            pred_adj = []
            n_edges = sum(adjacencies)
            for i in range(len(degrees)):
                for j in range(i):
                    pred_adj.append((degrees[i]*degrees[j])/((2*n_edges)-degrees[i]))
            all_pred_adj.append(pred_adj)
            all_adj.append(adjacencies)
        all_adj = np.asarray(all_adj)
        all_adj = all_adj.flatten()
        all_pred_adj = np.asarray(all_pred_adj)
        all_pred_adj = all_pred_adj.flatten()
        reorder = all_pred_adj.argsort()
        all_adj = all_adj[reorder]
        all_pred_adj = all_pred_adj[reorder]
        self.plot_ROC(all_adj,all_pred_adj,f'Chung-Lu ROC over {niter} iterations')
    
    #plot ROC curve for pred_adj sequence
    def plot_ROC(self,adj,pred_adj,title):
        aroc = 0.0
        TPRs = [0.0]
        FPRs = [0.0]
        n_pos = 0
        total_pos = sum(adj)
        total_neg = len(adj)-total_pos
        for i in range(len(pred_adj)-1,-1,-1):
            if adj[i]==1:#positive
                n_pos+=1
            TPRs.append(n_pos/total_pos)
            FPRs.append((len(pred_adj)-i-n_pos)/total_neg)
            aroc+=TPRs[-1]*(FPRs[-1]-FPRs[-2])
        plt.figure(figsize=(5,5))
        plt.plot(FPRs,TPRs)
        plt.xlabel('FPR')
        plt.ylabel('TPR')
        plt.title(title)
        plt.show()
        print(f'AROC: {aroc}')
        
    
    def save_model(self,path):
        self.model.save(path)
    
    def load_model(self,path):
        self.model = tf.keras.models.load_model(path)

In [6]:
print('reading. . .')
G = nx.read_graphml('mouse_retina_1.graphml')
G = G.to_undirected()#github says its undirected, and from/to is arbitrary, so ig do this
print('done')

reading. . .
done


In [29]:
cube_dim = 23
parse_dim = 3
n_epochs = 500
train_split = .5

In [30]:
mouse_connectome = Connectome(G)
generator = NullGenerator(mouse_connectome,cube_dim,parse_dim)

In [32]:
def subset_performance(subset,niter=500):#subset is boolean list [use degree, use clustering, use types]
    all_adj, all_pred_adj,clustering_diffs = [],[],[]
    for it in range(niter):
        if it%50==0:
            print(it)
        features, adjacencies, names, locations = [],[],[],[]
        while len(adjacencies)==0:
            features,adjacencies,names = mouse_connectome.restricted_get_and_parse(cube_dim,parse_dim,train_split,True,keep_all_inputs=True)
        
        if not subset[0]:#dont use degree
            for i in range(27):
                features[i] = np.random.uniform()
            features[27] = 0.0
        if not subset[1]:#dont use clustering
            features[28] = np.random.uniform()
            features[29] = 0.0
        if not subset[2]:#dont use types
            for i in range(30,len(features)-1):
                features[i] = np.random.uniform()
                features[-1] = 0.0
        features = np.expand_dims(features,0)
        predicted_adj = generator.model(features)[0]
        
        sorted_adj = np.copy(predicted_adj)
        sorted_adj = np.sort(sorted_adj)
        threshold = 0
        if sum(adjacencies)==0:
            threshold = sorted_adj[-1]+1.0
        else:
            threshold = sorted_adj[len(sorted_adj)-int(sum(adjacencies))]
        predicted_edges = []
        actual_edges = []
        for i in range(1,len(names)):
            adj_start = i*(i-1)/2
            for prev_node in range(i):
                adj_ind = int(prev_node+adj_start)
                if predicted_adj[adj_ind]>=threshold:
                    predicted_edges.append([names[i],names[prev_node]])
                if adjacencies[adj_ind]>.5:
                    actual_edges.append([names[i],names[prev_node]])
        true_subg = nx.Graph()
        true_subg.add_nodes_from(names)
        true_subg.add_edges_from(actual_edges)
        
        pred_subg = nx.Graph()
        pred_subg.add_nodes_from(names)
        pred_subg.add_edges_from(predicted_edges)
        
        clustering_diffs.append(abs(nx.algorithms.cluster.transitivity(true_subg)-nx.algorithms.cluster.transitivity(pred_subg)))
        all_pred_adj.append(predicted_adj.numpy())
        all_adj.append(adjacencies)

    #flatten and reorder arrays
    all_adj = np.asarray(all_adj)
    all_adj = all_adj.flatten()
    all_pred_adj = np.asarray(all_pred_adj)
    all_pred_adj = all_pred_adj.flatten()
    reorder = all_pred_adj.argsort()
    all_adj = all_adj[reorder]
    all_pred_adj = all_pred_adj[reorder]
    generator.plot_ROC(all_adj,all_pred_adj,f'Model ROC over {niter} iterations')
    print(f'average absolute difference in transitivity: {np.mean(clustering_diffs)}')
    return all_adj,all_pred_adj